In [75]:
import pandas as pd
import numpy as np

In [76]:
raw = pd.read_csv('Traffic_No_MODOT.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [77]:
#If the link_id has a link_id+1 present:
    #Then it has a pair (like east-west, or north-south shape pair) 
    #and is the first of the pair
#If the link_id does not have a link_id+1 present:
    #It is a second pair OR may be a street with only one traffic shape on it.
    #Could be a one-way, or other weird model fragments.
    #Deleting all of these results in missing streets
    #Keeping all of these results in a lot of duplicate line-segments
#If the link_Id+1 does not exist AND the link_id (not link_id+1) is odd, then it is a duplicate. 
    
#Let's operationalize this below.

In [78]:
#Create a new dataframe to preserve the old one
df = raw.copy()
#keep only the necessary columns
df = df[['link_id', 'volume', 'geometry']]

Let's first pull in the matching link_id's traffic volumes

In [79]:
#create a link_id_plus column
df['link_id_plus'] = df.link_id + 1

#Create a seperate dataset that we will join to the main df
dftemp = df.copy()
dftemp = dftemp[['link_id_plus', 'volume']]
#rename the link_id_plus since it needs to be the same in both datasets to merge on
dftemp.columns = ['link_id', 'volume']

#Okay, let's merge this back in to the original dataset
df = dftemp.merge(df, on='link_id', how='left')
#the merge sorts it weirdly, this makes it pretty again
df.sort_values(by=['link_id'], inplace=True)

Find if there is a link_id_Plus_One match

In [80]:
#Does the link_id_plus value exist as a link_id?
df["link_id_plus_exists"] = ""

#Use .isin to see if link_id_plus is in link_id. Returns true if it is.
df.link_id_plus_exists = df.link_id_plus.isin(df.link_id)

Is the link_id odd?

In [81]:
#create a link_id_is_odd column
df['link_id_is_even'] = ""

#Test if it is odd
df.link_id_is_even = (df.link_id & 1) == 0

Let's add in some summary details

In [84]:
#Average
df['mean_traffic_volume'] = df[["volume_x", "volume_y"]].mean(axis=1)
#StandardDeviation
df['standard_deviation_traffic_volume'] = df[["volume_x", "volume_y"]].std(axis=1)
#Replace all the NaNs with 
df.fillna("")

,link_id,volume_x,volume_y,geometry,link_id_plus,link_id_plus_exists,link_id_is_even,drop_this_observation,mean_traffic_volume,standard_deviation_traffic_volume
0,198483,184,114,"LINESTRING (-94.608064 38.9096912, -94.60793 3...",198484,True,False,Drop,149.0,49.4975
1,198484,114,,,,False,True,Keep,114.0,
2,198801,237,228,"LINESTRING (-94.6087852 39.0429951, -94.6086 3...",198802,True,False,Drop,232.5,6.36396
3,198802,228,,,,False,True,Keep,228.0,
4,299773,23,24,"LINESTRING (-94.6071638 39.1014239, -94.607081...",299774,True,False,Drop,23.5,0.707107
5,299774,24,,,,False,True,Keep,24.0,
6,303567,39,39,"LINESTRING (-94.60722320000001 39.101396, -94....",303568,True,False,Drop,39.0,0
7,303568,39,,,,False,True,Keep,39.0,
8,303799,515,558,"LINESTRING (-94.60712580000001 39.104445, -94....",303800,True,False,Drop,536.5,30.4056
9,303800,558,521,"LINESTRING (-94.60712580000001 39.104445, -94....",303801,True,True,Keep,539.5,26.163


Let's label every observation as drop or keep

In [82]:
#If link_id_plus_exists is TRUE  & link_id_is_even is TRUE  = keep it
#If link_id_plus_exists is FALSE & link_id_is_even is TRUE  = keep it
#If link_id_plus exists is FALSE & link_id_is_even is FALSE = Remove it
#if link_id_plus exists is True  & link_id_is_even is FALSE = Remove it

#Drop Column
df['drop_this_observation'] = ""

#This code takes the logic in the first cell comment and labels each observation as drop or keep.
df.drop_this_observation = np.where(
    ((df['link_id_plus_exists']==False) & (df['link_id_is_even']==False)) |
    ((df['link_id_plus_exists']==True) & (df['link_id_is_even']==False)), 'Drop','Keep')

Let's now drop all of the observations labeled Drop

In [87]:
df = df[df.drop_this_observation != 'Drop']

The df should now be clean enough to bring back into arcgis and merge it with the shapefile. 
If you use an "add join" and unselect all features it will only keep the ones in this dataset.
Join on the LinkID field.

Finally, we will export this as a csv

In [91]:
df.to_csv('final_clean_street_traffic_volume_segments.csv')